In [9]:
from diffusion_libs import *
from samples_generators import fill_vocabulary_c_v2, convert_back_to_code_c_v2, vocabulary_c_v2
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [10]:
fill_vocabulary_c_v2()
# sampling
min_signal_rate = 0.02
max_signal_rate = 0.95

# architecture
embedding_dims = 32
embedding_max_frequency = 1000.0
embedding_min_frequency = 1.0

# optimization
batch_size = 16
ema = 0.999
learning_rate = 1e-3

# dictionary related
DICTIONARY_SIZE = len(vocabulary_c_v2)
TOKENS_CAPACITY = 256

widths = [64, 64, 96, 128]
block_depth = 2

data_dir = f"./data/simple_c_v2/"
lang_base = f"checkpoints/simple_c_v2"
model_path = f"E:\Studies\master_thesis\diffusion_models_checpoints_savespace\D16-02-2023\simple_c_v2\cp-0011\model"

Dictionary already filled


In [11]:
network = get_network(
      TOKENS_CAPACITY, embedding_min_frequency, embedding_max_frequency, 
      embedding_dims, widths=widths, block_depth=block_depth, name="complicated"
  )

model = DiffusionModel(
      TOKENS_CAPACITY, DICTIONARY_SIZE, network, batch_size, max_signal_rate, 
      min_signal_rate, ema, False
  )

model.compile(
    optimizer = keras.optimizers.experimental.Adam(
        learning_rate=learning_rate
    ),
    loss = keras.losses.mean_absolute_error
)

#normalizer
n_w = np.load(f"{lang_base}/normalizer_weights.npy", allow_pickle=True)
normalizer = keras.layers.Normalization(mean=n_w[0], variance=n_w[1])
normalizer.build((TOKENS_CAPACITY))
model.normalizer = normalizer
model.load_weights(model_path)

### First should generate some sample to see if it still generates something that looks like code

In [14]:
raw, denormalized = model.generate(5,100)
for sample in denormalized:
  scaled = scale_dataset(sample, DICTIONARY_SIZE)
  print(scaled)
  print(" ".join(convert_back_to_code_c_v2(scaled)).replace("EMPTY", "").replace(";", ";\n").replace("{", "{\n"))
  print()

[ 1.0388548 27.642866  15.357128   6.593502  28.433435  17.50827
 16.804281  25.514482  20.105173  12.459578  18.851442  21.685682
 12.004677  19.234154  18.174858  18.196083  12.95696   22.956282
 18.731342  14.833598  22.288784  18.622084  16.063412  19.043709
 15.72563   23.08742   19.031223  23.197954  18.340431  16.622599
 19.42147   16.707851  19.743282  16.397436  23.025103  19.51
 22.478773  18.682268  16.69658   19.958307  17.590433  20.113565
 19.443092  19.824997  18.890167  18.009384  18.746979  23.675957
 19.636896  17.075869  20.299904  21.441174  19.86877   23.292885
 19.253609  20.176184  19.958414  23.61001   20.163822  18.318916
 19.34182   17.680193  20.489836  20.39944   20.963745  21.734976
 19.99455   19.37787   21.06434   21.196539  21.539623  21.3088
 21.375093  21.275461  21.431562  21.430487  21.40341   21.7054
 21.546036  21.541426  21.45748   21.529264  21.682707  21.6687
 21.461634  21.724977  21.650103  21.746763  21.514624  21.864939
 21.725634  21.744497